In [ ]:
# import sys
# !{sys.executable} -m pip install matplotlib --user
# !{sys.executable} -m pip install hyperopt --user

In [1]:
from time import time
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.neural_network import MLPClassifier
import numpy as np

# Data science imports
import pandas as pd
from enum import Enum
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn_pandas import DataFrameMapper

# Visualisation imports
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# File IO: Read the data file
file = open('train_data.txt', mode = 'r')
lines = file.readlines()
file.close()

In [3]:
# Check for completeness
len(lines)

78945

In [4]:
# Pre-processing: Clean unwanted characters 
data = []
i = 0
while i < len(lines)-1:
    row = {}
    row['id'] = lines[i+1].split(" ")[1].strip()
    row['durationOfStay'] = lines[i+2].split(" ")[1].strip()
    row['gender'] = lines[i+3].split(" ")[1].strip()
    row['Age'] = lines[i+4].split(" ")[1].strip()
    row['kids'] = lines[i+5].split(" ")[1].strip()
    row['destinationCode'] = lines[i+6].split(" ")[1].strip()
    row['AcomType'] = lines[i+7].split(" ")[1].strip()
    data.append(row)
    i = i + 8

In [5]:
# Structure the data
def structure_data_to_df(data, test_data=False):
    if test_data:
        df = pd.DataFrame(data, columns=['id','durationOfStay','gender','Age','kids','destinationCode'])
        df['id'] = df['id'].astype(str)
    else:
        df = pd.DataFrame(data, columns=['durationOfStay','gender','Age','kids','destinationCode','AcomType'])
        df['AcomType'] = df['AcomType'].astype(str)
        
    df['gender'] = df['gender'].astype(str)
    df['kids'] = df['kids'].astype(str)
    df['destinationCode'] = df['destinationCode'].astype(str)
    df = df.replace('<NA>', np.nan)
    df = df.replace('NA', np.nan)
    df = df.replace('nan', np.nan)
    df[['durationOfStay', 'Age']] = df[['durationOfStay', 'Age']].apply(pd.to_numeric)
    
    return df
    
df = structure_data_to_df(data)

In [6]:
# utility variables
categorical_cols = ['gender']
missing_categorical_cols = ['kids', 'destinationCode']
numerical_cols = ['durationOfStay']
missing_numerical_cols = ['Age']

# transform utility function
def build_imputer_and_transformer(missing_categorical_cols, categorical_cols, 
                                  missing_numerical_cols, numerical_cols, constant=True):
    transformations = []
    
    for col in missing_categorical_cols:
        if constant:
            transformations.append(([col], [SimpleImputer(strategy='constant', fill_value='N/A'), LabelBinarizer()]))
        else:
            transformations.append(([col], [SimpleImputer(strategy='most_frequent'), LabelBinarizer()]))
        
    for col in categorical_cols:
        transformations.append((col, LabelBinarizer()))
        
    transformations.append((missing_numerical_cols, [SimpleImputer(strategy='mean'), StandardScaler()]))
    transformations.append((numerical_cols, StandardScaler()))
   
    transformer = DataFrameMapper(transformations)
    return transformer

In [7]:
y = df['AcomType'].copy()
X = df.drop(['AcomType'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
    
transformer = build_imputer_and_transformer(missing_categorical_cols, categorical_cols,
                                             missing_numerical_cols, numerical_cols, constant=True)
    
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)
    

In [8]:
criterion_list = ['gini', 'entropy']
n_estimators_list = np.arange(10, 1000, 10, dtype=int)
max_features_list = np.arange(1, X.shape[1], 1, dtype=int) 
max_depth_list = np.arange(1, 50, 1, dtype=int) 
min_samples_split_list = np.arange(10, 100, 10, dtype=int)
min_samples_leaf_list = np.arange(1, 30, 1, dtype=int)
max_leaf_nodes_list = np.arange(10, 30, 2, dtype=int)
class_weights = [None, 'balanced']
bootstrap_list =[True, False]

In [9]:
def optimize(evals, trials, optimizer=tpe.suggest):
    space = {
        'criterion': hp.choice('criterion', criterion_list),
        'n_estimators': hp.choice('n_estimators', n_estimators_list),
        'max_features': hp.choice('max_features', max_features_list),
        'max_depth': hp.choice('max_depth', max_depth_list),
        'min_samples_split': hp.choice('min_samples_split', min_samples_split_list),
        'min_samples_leaf': hp.choice('min_samples_leaf', min_samples_leaf_list),
        'max_leaf_nodes': hp.choice('max_leaf_nodes', max_leaf_nodes_list),
        'class_weight': hp.choice('class_weight', class_weights),
        'bootstrap': hp.choice('bootstrap', bootstrap_list)
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [10]:
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

def score(params):
    criterion = params['criterion']
    n_estimators = params['n_estimators']
    max_features = params['max_features']
    max_depth = params['max_depth']
    min_samples_split = params['min_samples_split']
    min_samples_leaf = params['min_samples_leaf']
    max_leaf_nodes = params['max_leaf_nodes']
    class_weight = params['class_weight']
    bootstrap = params['bootstrap']
    
    model = RandomForestClassifier(criterion=criterion, n_estimators=n_estimators, max_features=max_features,
                                  max_depth=max_depth, min_samples_split=min_samples_split, 
                                   min_samples_leaf=min_samples_leaf,
                                  max_leaf_nodes=max_leaf_nodes, class_weight=class_weight, bootstrap=bootstrap,
                                  random_state=42, n_jobs=48)
    
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='balanced_accuracy', n_jobs=40)  
    
    scores_mean = 1.0 - scores.mean()

    
    print('Parameters with this training loss {} :'.format(scores_mean))
    print(params)
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': scores_mean, 'status': STATUS_OK}
     

In [11]:
import time
from tqdm import tqdm
import numpy as np

trials = Trials()
cores = 48
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
best_param['criterion'] = criterion_list[best_param['criterion']]
best_param['max_leaf_nodes'] = max_leaf_nodes_list[best_param['max_leaf_nodes']]
best_param['min_samples_leaf'] = min_samples_leaf_list[best_param['min_samples_leaf']]
best_param['n_estimators'] = n_estimators_list[best_param['n_estimators']]
best_param['max_features'] = max_features_list[best_param['max_features']]
best_param['min_samples_split'] = min_samples_split_list[best_param['min_samples_split']]
best_param['max_depth'] = max_depth_list[best_param['max_depth']]
best_param['class_weight'] = class_weights[best_param['class_weight']]
best_param['bootstrap'] = bootstrap_list[best_param['bootstrap']]
print('\n Best score:')
score(best_param)

Hyperopt:   0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
Parameters with this training score 0.7671526944389784 :

  0%|          | 0/1000 [00:03<?, ?it/s, best loss: ?]
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 41, 'max_features': 4, 'max_leaf_nodes': 24, 'min_samples_leaf': 3, 'min_samples_split': 90, 'n_estimators': 540}

  0%|          | 0/1000 [00:03<?, ?it/s, best loss: ?]

Hyperopt:   0%|          | 1/1000 [00:04<1:06:39,  4.00s/it]


  0%|          | 1/1000 [00:03<1:04:35,  3.88s/it, best loss: 0.23284730556102162]
Parameters with this training score 0.7492524575542107 :

  0%|          | 1/1000 [00:05<1:04:35,  3.88s/it, best loss: 0.23284730556102162]
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 47, 'max_features': 4, 'max_leaf_nodes': 12, 'min_samples_leaf': 28, 'min_samples_split': 90, 'n_estimators': 390}

  0%|          | 1/1000 [00:05<1:04:35,  3.88s/it, best loss: 0.23284730556102162]

Hyperopt:   0%|          | 2/1000 [00:06<56:56,  3.42s/it]  

...


100%|█████████▉| 999/1000 [29:43<00:01,  1.86s/it, best loss: 0.2197485425425898]
Parameters with this training score 0.5129734681576275 :

100%|█████████▉| 999/1000 [29:44<00:01,  1.86s/it, best loss: 0.2197485425425898]
{'bootstrap': False, 'class_weight': None, 'criterion': 'gini', 'max_depth': 21, 'max_features': 3, 'max_leaf_nodes': 16, 'min_samples_leaf': 27, 'min_samples_split': 50, 'n_estimators': 360}

100%|█████████▉| 999/1000 [29:44<00:01,  1.86s/it, best loss: 0.2197485425425898]

Hyperopt: 100%|██████████| 1000/1000 [29:44<00:00,  1.73s/it]


100%|██████████| 1000/1000 [29:44<00:00,  1.73s/it, best loss: 0.2197485425425898]------------------------------------
The best hyperparameters are:  

{'bootstrap': 1, 'class_weight': 1, 'criterion': 0, 'max_depth': 25, 'max_features': 2, 'max_leaf_nodes': 9, 'min_samples_leaf': 26, 'min_samples_split': 8, 'n_estimators': 34}
Time elapsed to optimize 1000 executions: 1784.6413373947144

 Best score:


Parameters with this training score 0.7802514574574102 :
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 26, 'max_features': 3, 'max_leaf_nodes': 28, 'min_samples_leaf': 27, 'min_samples_split': 90, 'n_estimators': 350}


{'loss': 0.2197485425425898, 'status': 'ok'}

In [ ]:
best_param=

In [12]:
# get the best params and train the model
RF_opt = RandomForestClassifier(criterion=best_param['criterion'], n_estimators=best_param['n_estimators'], max_features=best_param['max_features'],
                                  max_depth=best_param['max_depth'], min_samples_split=best_param['min_samples_split'], min_samples_leaf=best_param['min_samples_leaf'],
                                  max_leaf_nodes=best_param['max_leaf_nodes'], class_weight=best_param['class_weight'], bootstrap=best_param['bootstrap'],
                                  random_state=42, n_jobs=48)


fitted_model = RF_opt.fit(X=X_train,y=y_train)

In [13]:
y_score = fitted_model.predict_proba(X_test)
y_score

array([[0.47585445, 0.52414555],
       [0.57627697, 0.42372303],
       [0.43789423, 0.56210577],
       ...,
       [0.43938594, 0.56061406],
       [0.51392128, 0.48607872],
       [0.50080173, 0.49919827]])

In [14]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc

0.7720364741641338

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline 
x_list =range(len(train_acc_list))

plt.plot(x_list, train_acc_list, label='Training accuracy')
plt.plot(x_list, test_acc_list, label='Testing accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_list, train_loss_list, label='Training loss')
plt.plot(x_list, test_loss_list, label='Testing loss')
plt.legend()
plt.show()